In [ ]:

import pyrealsense2 as rs
import numpy as np
import os.path
import matplotlib.pyplot as plt
import open3d as o3d
from sympy import python

In [ ]:
n = 9
point_cloud = o3d.io.read_point_cloud(f"C:/Users/mhmon/ovos-pv/Bandeja/frescos/{n}.ply")


In [ ]:
# ler nuvem de pontos
# o3d.visualization.draw_geometries([point_cloud])
inicial = 0
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        point_cloud.cluster_dbscan(eps=0.002, min_points=10, print_progress=True))

# quantidade de clusters
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

# Obter a quantidade de pontos por cluster
cluster_sizes = [np.sum(labels == i) for i in range(max_label + 1)]
print(cluster_sizes)

# filtrar nuvem de pontos por tamanho do cluster
# min_cluster_size = 1100
# max_cluster_size = 5200
min_cluster_size = 500
max_cluster_size = 10000
filtered_clusters = [i for i in range(max_label + 1) if cluster_sizes[i] > min_cluster_size and cluster_sizes[i] < max_cluster_size]

# armazena valores que estao presentes ao mesmo tempo em labels e filtered_clusters
filtered_points = np.isin(labels, filtered_clusters)
# gera nuvem de pontos
filtered_pcd = point_cloud.select_by_index(np.where(filtered_points)[0])

#obter labels que restaram após filtragem
filtered_labels = labels[filtered_points]
max_filtered_label = filtered_labels.max()

# Obter a quantidade de pontos por cluster

new_cluster_sizes = [np.sum(filtered_labels == i) for i in range(max_filtered_label + 1)]
# print(new_cluster_sizes)
# print(f"point cloud has {max_filtered_label + 1} clusters")


# # Usar um mapa de cores para os clusters filtrados
colors = plt.get_cmap("tab20")(filtered_labels / (max_filtered_label if max_filtered_label > 0 else 1))
colors[filtered_labels <= 0] = 0  # Colore o ruído de preto (se houver)

# Atribuir as cores à nuvem de pontos filtrada
filtered_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])


# Visualizar a nuvem de pontos filtrada sem o ruído
o3d.visualization.draw_geometries([filtered_pcd])

# separar os labels de cada cluster (ovo)
unique_labels = np.unique(filtered_labels)



from sklearn.decomposition import PCA

# Função para calcular a excentricidade do cluster usando PCA
def calcular_excentricidade(pcd):
    pontos = np.asarray(pcd.points)
    pca = PCA(n_components=3)
    pca.fit(pontos)
    # Excentricidade é a razão entre os comprimentos do maior e menor eixo
    excentricidade = pca.explained_variance_ratio_[0] / pca.explained_variance_ratio_[-1]
    return excentricidade

# Filtrar clusters com base na excentricidade
new_filtered_clusters = []
limite_excentricidade =  20 # ajustável conforme os testes

for label in unique_labels:
    if label < 0:
        continue
    cluster_indices = np.where(filtered_labels == label)[0]
    cluster_pcd = filtered_pcd.select_by_index(cluster_indices)

    # Calcular excentricidade
    excentricidade = calcular_excentricidade(cluster_pcd)
    print(excentricidade)
    # # Clusters com excentricidade baixa são mantidos
    if (excentricidade < limite_excentricidade):
        new_filtered_clusters.append(label)

centroides = []
for label in new_filtered_clusters:
    cluster_indices = np.where(filtered_labels == label)[0]
    cluster_pcd = filtered_pcd.select_by_index(cluster_indices)
    pontos = np.asarray(cluster_pcd.points)
    centroide = pontos.mean(axis=0)
    centroides.append((label, centroide))

# Função para ordenar os clusters conforme a disposição da bandeja (colunas de cima pra baixo)
def ordenar_clusters_por_coluna(centroides, n_linhas=3, n_colunas=5):
    # Ordenar por X (horizontal) para obter as colunas
    centroides = sorted(centroides, key=lambda x: x[1][0])
    # Agrupar em colunas
    colunas = [centroides[i * n_linhas:(i + 1) * n_linhas] for i in range(n_colunas)]
    # Ordenar cada coluna por Y decrescente
    for i in range(n_colunas):
        colunas[i] = sorted(colunas[i], key=lambda x: -x[1][1])
    # Reconstruir lista ordenada
    return [label for coluna in colunas for label, _ in coluna]

# Obter lista final de labels ordenados
ordered_labels = ordenar_clusters_por_coluna(centroides)

print(new_filtered_clusters.__len__())


In [ ]:
value = 121
if len(ordered_labels) == 15 or len(ordered_labels) == 3:
    for i, label in enumerate(ordered_labels):
        nome = value + i
        if nome == 786:
            nome+=1
            value+=1
        cluster_indices = np.where(filtered_labels == label)[0]
        cluster_pcd = filtered_pcd.select_by_index(cluster_indices)
        o3d.io.write_point_cloud(f"C:/Users/mhmon/ovos-pv/ovos3/ovo{nome}.ply" , cluster_pcd)
        print(nome)
else:
  #caso haja ovos a mais ou a menos, visualizar índices para remover (geralmente o primeiro cluster ou o último)
    for i, label in enumerate(ordered_labels):
        cluster_indices = np.where(filtered_labels == label)[0]
        cluster_pcd = filtered_pcd.select_by_index(cluster_indices)
        print(i)
        o3d.visualization.draw_geometries([cluster_pcd])

121
122
123
124
125
126
127
128
129
130
131
132
133
134
135


In [ ]:
# indices_to_remove = [0,1,7,8, 14, 15, 21, 22]
# for index in sorted(indices_to_remove, reverse=True):
#     del new_filtered_clusters[index]


centroides = []
for label in new_filtered_clusters:
    cluster_indices = np.where(filtered_labels == label)[0]
    cluster_pcd = filtered_pcd.select_by_index(cluster_indices)
    pontos = np.asarray(cluster_pcd.points)
    centroide = pontos.mean(axis=0)
    centroides.append((label, centroide))

ordered_labels = ordenar_clusters_por_coluna(centroides)
print(ordered_labels.__len__())


15


In [ ]:
for i, label in enumerate(ordered_labels):
        cluster_indices = np.where(filtered_labels == label)[0]
        cluster_pcd = filtered_pcd.select_by_index(cluster_indices)
        print(i)
        o3d.visualization.draw_geometries([cluster_pcd])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [ ]:
for i, label in enumerate(new_filtered_clusters):
        cluster_indices = np.where(filtered_labels == label)[0]
        cluster_pcd = filtered_pcd.select_by_index(cluster_indices)
        print(i)
        o3d.visualization.draw_geometries([cluster_pcd])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
